# Part I. ETL Pipeline for Pre-Processing the Files

In Part I, the existing files are preprocessed and a new file, event_datafile_new.csv, is created for further processing.

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Loading the data into Apache Cassandra and running queries 

The CSV file titled <font color=red>event_datafile_new.csv</font> crete in Part I contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Creating an Apache Cassandra and a dedicated keyspace

In [2]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [3]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [4]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Creating tables and running analyical queries

Cassandra requires separate tables with dedicated schemas to answer queries. The following cells create tables and import the corresponding data to answer queries of these three kinds:

- Question 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
- Question 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182   
- Question 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Question 1 : Give me the artist, song title and song's length in the music app history that was heard during sessionId=338, itemInSession=4

Apache Cassandra is a NoSQL database with a denormalized schema that is heavily optimized for write performance. To retrieve data from the database, the fields being queried for must be part of a table's primary key. This in turn means you should follw a "one table per query" approach, where the table schema is designed to allow for specific queries.
In the case of question 1, the user wants to
- retrieve the artist name, song title and song length
- for a given sessionId and itemInSession
Therefore, all five fields should be included in the table schema, with the sessionId and itemInSession field composing the primary (partition) key. In the following statements, a table with this schema is created specifically for answering questions of type 1. The data is then loaded into the table and the query issued.

In [23]:
query1 = "CREATE TABLE IF NOT EXISTS session_plays "
query1 = query1 + "(session_id int, item_in_session int, artist_name text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session));"
try:
    session.execute(query1)
except Exception as e:
    print(e)                    

In [24]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_plays (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s);"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### Answer 1

In [25]:
query = "SELECT artist_name, song_title, song_length from session_plays WHERE session_id=338 AND item_in_session=4;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, "-", row.song_title, "-", row.song_length)

Faithless - Music Matters (Mark Knight Dub) - 495.30731201171875


### Question 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Again following the "one table per query" approach, another dedicated table is created and the data loaded into it. In addition to the two fields searched by (user_id and session_id) forming the partition key, and the data fields for artist_name and song_title being included, a specific sort order (by item_in_session) for the query result is requested. This can be achieved by extending the primary key with a clustering column for this field.

In [26]:
query2 = "CREATE TABLE IF NOT EXISTS user_plays "
query2 = query2 + "(user_id int, session_id int, item_in_session int, artist_name text, song_title text, first_name text, last_name text, PRIMARY KEY ((user_id,session_id), item_in_session)) "
query2 = query2 + "WITH CLUSTERING ORDER BY (item_in_session ASC);"
try:
    session.execute(query2)
except Exception as e:
    print(e)                   

In [27]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_plays (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

### Answer 2

In [30]:
query = "SELECT artist_name, song_title, first_name, last_name from user_plays WHERE user_id=10 AND session_id=182;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, "-", row.song_title, "-", row.first_name, row.last_name)

Down To The Bone - Keep On Keepin' On - Sylvie Cruz
Three Drives - Greece 2000 - Sylvie Cruz
Sebastien Tellier - Kilometer - Sylvie Cruz
Lonnie Gordon - Catch You Baby (Steve Pitron & Max Sanna Radio Edit) - Sylvie Cruz


### Question 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own

Another query, another table. Since the combination of user_id and song_title is sufficient and required to uniquely identify a user listening to a particular song, those two field form the primary key. The user's first_name and last_name are included as data fields in the search result. However, Apache Cassandra was built for Big Data applications, and care has to be taken when designing the queries. For this question, only a part of the primary key (song_title) is used in the WHERE clause, with the user_id not being specified. The primary key is therefore not completely specified and Cassandra can't use it for the query. One option would be to retrieve *all* rows and then filtering for the song title (ALLOW FILTERING), but could easily overflow the client's memory and/or perform very badly. A better approach is to create an index for the song title, letting Cassandra to easily execute the query. Since the index will only use the song title, the index size should be manageable.

In [31]:
query3 = "CREATE TABLE IF NOT EXISTS songs "
query3 = query3 + "(user_id int, song_title text, first_name text, last_name text, PRIMARY KEY (user_id, song_title));"
query4 = "CREATE INDEX song_title_idx ON songs (song_title);"

try:
    session.execute(query3)
    session.execute(query4)
except Exception as e:
    print(e)

                    

Error from server: code=2200 [Invalid query] message="Index song_title_idx already exists"


In [18]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs (user_id, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s);"
        session.execute(query, (int(line[10]), line[9], line[1], line[4]))

### Answer 3

In [32]:
query = "SELECT user_id, first_name, last_name from songs WHERE song_title='All Hands Against His Own';"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Sara Johnson
Tegan Levine
Jacqueline Lynch


### Cleaning up by dropping the tables

In [22]:
try:
    session.execute('DROP TABLE session_plays;')
    session.execute('DROP TABLE user_plays;')
    session.execute('DROP TABLE songs;')
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="unconfigured table session_plays"


### Close the session and cluster connection¶

In [64]:
session.shutdown()
cluster.shutdown()